# ADVERSARIAL AUTOENCODER PARA CLASIFICACIÓN SEMI SUPERVISADA

Un problema de clasificacion multiclase de imagenes puede resultar una tarea desafiante si no se cuenta con muchos datos etiquetados. En el paper "Adversarial Autoencoders" de 2016 se propone una solución. Los Adversarial Autoencoders (AAEs) ofrecen una alternativa eficiente para realizar inferencia variacional, integrando un autoencoder clásico con una componente adversarial inspirada en las Generative Adversarial Networks (GANs).
Algunas aplicaciones son la clasificación semi-supervisada, análisis de representación y tareas de generación.
En este trabajo nos enfocamos en su aplicación sobre el conjunto de datos MNIST y exploramos sus capacidades para aprendizaje estructurado con datos parcialmente etiquetados.

# PAQUETES


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
from torch.optim.lr_scheduler import MultiStepLR

# CONFIGURACION Y DATOS


In [ ]:
batch_size = 100
z_dim = 10  #cant de "estilos"
y_dim = 10  # cant de clases (etiquetasreales del mnist)
h_dim = 1000  # tamaño de capas ocultas
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Usamos el dataset mnist
transform = transforms.Compose([transforms.ToTensor()])
mnist_train = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# Hacemos el dataset para todo lo que es semi supervizado
np.random.seed(202506)
labels_porclase = 20
indices = []
for c in range(10):
    idx = np.where(np.array(mnist_train.targets) == c)[0]
    indices.extend(np.random.choice(idx, labels_porclase, replace=False))

subconj_etiquetado = Subset(mnist_train, indices)
idx_sin_etiqueta = list(set(range(len(mnist_train))) - set(indices))
subjconj_sin_etiqueta = Subset(mnist_train, idx_sin_etiqueta)

labeled_loader = DataLoader(subconj_etiquetado, batch_size=batch_size, shuffle=True)
unlabeled_loader = DataLoader(subjconj_sin_etiqueta, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)


In [ ]:
# esto es una prueba aparte

import numpy as np
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms

# ---------------------- PARÁMETROS ----------------------
modo = "por_clase"  # "por_clase" o "porcentaje"
labels_por_clase = 100  # solo se usa si modo == "por_clase"
porcentaje_etiquetado = 0.05  # solo se usa si modo == "porcentaje"
batch_size = 64
semilla = 202506

# ---------------------- CARGA DEL DATASET ----------------------
transform = transforms.Compose([transforms.ToTensor()])
mnist_train = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# ---------------------- CREACIÓN SEMISUPERVISADA ----------------------
np.random.seed(semilla)
targets = np.array(mnist_train.targets)

if modo == "por_clase":
    indices_etiquetados = []
    for c in range(10):
        idx_clase = np.where(targets == c)[0]
        indices_etiquetados.extend(np.random.choice(idx_clase, labels_por_clase, replace=False))

elif modo == "porcentaje":
    total_etiquetados = int(len(mnist_train) * porcentaje_etiquetado)
    indices_etiquetados = np.random.choice(len(mnist_train), total_etiquetados, replace=False).tolist()

else:
    raise ValueError("El modo debe ser 'por_clase' o 'porcentaje'.")

indices_sin_etiqueta = list(set(range(len(mnist_train))) - set(indices_etiquetados))

# ---------------------- CONJUNTOS DE DATOS ----------------------
subconj_etiquetado = Subset(mnist_train, indices_etiquetados)
subconj_sin_etiqueta = Subset(mnist_train, indices_sin_etiqueta)

labeled_loader = DataLoader(subconj_etiquetado, batch_size=batch_size, shuffle=True)
unlabeled_loader = DataLoader(subconj_sin_etiqueta, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)


# MODELO

Consideremos la siguiente arquitectura, donde el encoder genera dos vectores: y, el vector que representa las clases, y z, el vector que representa diferentes estilos de escritura. El decoder es entrenado para reconstruir la imagen, mientras que el encoder se entrena para generar buenas representaciones del espacio latente. Un discriminador Dy fuerza a que el vector y sea categórico, mientras que un discriminador Dz fuerza a que el vector z siga una distribución gaussiana con desviación estándar 1.

## ENCODER

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(784, h_dim), nn.ReLU(),
           nn.Linear(h_dim, h_dim), nn.ReLU()
        )
        self.fc_z = nn.Linear(h_dim, z_dim)         # vector del "estilo" (este se va aprendiendo solo)
        self.fc_y = nn.Linear(h_dim, y_dim)         # Clase



    def forward(self, x):
        x = x.view(-1, 784)
        h = self.shared(x)
        z = self.fc_z(h)
        y = F.softmax(self.fc_y(h), dim=1)
        return z, y


## DECODER



In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(z_dim + y_dim, h_dim), nn.ReLU(),
            nn.Linear(h_dim, h_dim), nn.ReLU(),
            nn.Linear(h_dim, 784), nn.Sigmoid()
        )

    def forward(self, z, y):
        x_hat = self.fc(torch.cat([z, y], dim=1))
        return x_hat.view(-1, 1, 28, 28)

## PROPUESTA ALTERNATIVA PARA ENCODER Y DECODER


Consideramos que 1000 neuronas en la shidden layers puede ser un numero muy alto para aprender solo 10 clases y que la red sea propensa a sobreajustar. Proponemos una arquitectura alternativa mas relacionada a un autoencoder clasico con capas que van bajando progresivamente la dimensionalidad en el encoder y aumentandola en el decoder.

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(784, 500), nn.ReLU(),
            nn.Linear(500, 300), nn.ReLU()
        )
        self.fc_z = nn.Linear(300, z_dim)
        self.fc_y = nn.Linear(300, y_dim)

    def forward(self, x):
        x = x.view(-1, 784)
        h = self.shared(x)
        z = self.fc_z(h)
        y = F.softmax(self.fc_y(h), dim=1)
        return z, y


class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim + y_dim, 300), nn.ReLU(),
            nn.Linear(300, 500), nn.ReLU(),
            nn.Linear(500, 784), nn.Sigmoid()
        )

    def forward(self, z, y):
        return self.net(torch.cat([z, y], dim=1)).view(-1, 1, 28, 28)

## DISCRIMINADOR Z E Y

In [ ]:
class DiscriminadorZ(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, h_dim), nn.ReLU(),
            nn.Linear(h_dim, 1), nn.Sigmoid()
        )

    def forward(self, z):
        return self.net(z)

class DiscriminadorY(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(y_dim, h_dim), nn.ReLU(),
            nn.Linear(h_dim, 1), nn.Sigmoid()
        )

    def forward(self, y):
        return self.net(y)

#INICIALIZACION


Estudiamos hiperparametros y decidimos realizar algunas modificaciones con respecto al paper. Vemos que la red tiene muy buenos resultados luego de 30 epochs, asi que vamos ajustando el elarning rate de forma acorde.

In [ ]:
encoder = Encoder().to(device)
decoder = Decoder().to(device)
dz = DiscriminadorZ().to(device)
dy = DiscriminadorY().to(device)

recon_optimizador = optim.SGD(list(encoder.parameters()) + list(decoder.parameters()), lr=0.01, momentum=0.9)
dz_optimizador = optim.SGD(dz.parameters(), lr=0.1, momentum=0.1)
dy_optimizador = optim.SGD(dy.parameters(), lr=0.1, momentum=0.5)
class_optimizador = optim.SGD(encoder.parameters(), lr=0.1, momentum=0.9)


# learning rates según el paper
recon_scheduler = MultiStepLR(recon_optimizador, milestones=[30, 60, 90, 120], gamma=0.1)
class_scheduler = MultiStepLR(class_optimizador, milestones=[30, 60, 90, 120], gamma=0.1)
dz_scheduler = MultiStepLR(dz_optimizador, milestones=[30, 60, 90, 120], gamma=0.1)
dy_scheduler = MultiStepLR(dy_optimizador, milestones=[30, 60, 90, 120], gamma=0.1)

# (el momentum esta definido con esos valores para copiar lo del paper)

# ENTRENAMIENTO

Entrenamos por 150 epochs y ajustamos la loss del discriminador z y de la clasificacion para mejorar el rendimiento que buscamos.

In [ ]:
for epoch in range(2500):
    encoder.train()
    decoder.train()

    for (ul_x, _), (l_x, l_y) in zip(unlabeled_loader, labeled_loader):
        ul_x = ul_x.to(device)
        ul_x = ul_x + 0.3 * torch.randn_like(ul_x)
        l_x = l_x.to(device)
        l_y = l_y.to(device)

        #Reconstruccion
        z, y = encoder(ul_x)
        x_hat = decoder(z, y)
        loss_recon = F.mse_loss(x_hat, ul_x)
        recon_optimizador.zero_grad()
        loss_recon.backward()
        recon_optimizador.step()



        #Discriminador Z
        z_real = torch.randn(ul_x.size(0), z_dim).to(device)
        z_fake, _ = encoder(ul_x)
        dz_real = dz(z_real)
        dz_fake = dz(z_fake.detach())
        loss_dz = -torch.mean(torch.log(dz_real + 1e-8) + torch.log(1 - dz_fake + 1e-8))
        loss_dz = 2 * loss_dz
        dz_optimizador.zero_grad()
        loss_dz.backward()
        dz_optimizador.step()

        # Discriminador Y
        y_real = F.one_hot(torch.randint(0, y_dim, (ul_x.size(0),)), num_classes=y_dim).float().to(device)
        _, y_fake = encoder(ul_x)
        dy_real = dy(y_real)
        dy_fake = dy(y_fake.detach())
        loss_dy = -torch.mean(torch.log(dy_real + 1e-8) + torch.log(1 - dy_fake + 1e-8))
        dy_optimizador.zero_grad()
        loss_dy.backward()
        dy_optimizador.step()

        # Clasificacion (supervisada)
        _, y_pred = encoder(l_x)
        loss_cls = 2.0 * F.cross_entropy(y_pred, l_y)
        class_optimizador.zero_grad()
        loss_cls.backward()
        class_optimizador.step()

    print(f"Epoch {epoch} | Recon: {loss_recon.item():.4f} | Dz: {loss_dz.item()/2:.4f} | Dy: {loss_dy.item():.4f} | Cls: {loss_cls.item()/2:.4f}")
    if epoch % 10 == 0:
        encoder.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for x_test, y_test in test_loader:
                x_test, y_test = x_test.to(device), y_test.to(device)
                _, y_pred = encoder(x_test)
                pred_labels = torch.argmax(y_pred, dim=1)
                correct += (pred_labels == y_test).sum().item()
                total += y_test.size(0)
        acc = 100 * correct / total
        print(f"Test Accuracy: {acc:.2f}%")
    recon_scheduler.step()
    class_scheduler.step()
    dz_scheduler.step()
    dy_scheduler.step()



Epoch 0 | Recon: 0.3222 | Dz: 0.1479 | Dy: 0.7786 | Cls: 2.2934
Test Accuracy: 38.12%
Epoch 1 | Recon: 0.3244 | Dz: 0.0812 | Dy: 0.4719 | Cls: 2.2111
Epoch 2 | Recon: 0.3228 | Dz: 0.0771 | Dy: 1.0719 | Cls: 1.7873
Epoch 3 | Recon: 0.3188 | Dz: 0.0613 | Dy: 1.1415 | Cls: 1.8353
Epoch 4 | Recon: 0.3181 | Dz: 0.0567 | Dy: 1.0689 | Cls: 1.7291
Epoch 5 | Recon: 0.3183 | Dz: 0.0526 | Dy: 1.0735 | Cls: 1.7489
Epoch 6 | Recon: 0.3191 | Dz: 0.0387 | Dy: 1.0748 | Cls: 1.8755
Epoch 7 | Recon: 0.3166 | Dz: 0.0194 | Dy: 1.1496 | Cls: 1.7855
Epoch 8 | Recon: 0.3162 | Dz: 0.0345 | Dy: 1.1186 | Cls: 1.8840
Epoch 9 | Recon: 0.3120 | Dz: 0.0249 | Dy: 1.0219 | Cls: 1.7584
Epoch 10 | Recon: 0.3139 | Dz: 0.0245 | Dy: 1.0230 | Cls: 1.7308
Test Accuracy: 66.36%
Epoch 11 | Recon: 0.3117 | Dz: 0.0238 | Dy: 1.0810 | Cls: 1.7805
Epoch 12 | Recon: 0.3037 | Dz: 0.0491 | Dy: 1.1095 | Cls: 1.8794
Epoch 13 | Recon: 0.2962 | Dz: 0.0042 | Dy: 1.1197 | Cls: 1.8042
Epoch 14 | Recon: 0.2881 | Dz: 0.0036 | Dy: 1.0417 | Cls

# CLASIFICACION DE NUMERO DIBUJADO

El siguiente codigo es para probar el modelo con un numero dibujado a mano.

## Preprocesamiento de imagen

In [ ]:
from PIL import Image, ImageOps, ImageFilter, ImageEnhance
import torchvision.transforms as T
import torch

def procesar_imagen(foto):
    imagen = Image.open(foto).convert("L")
    imagen = ImageOps.invert(imagen)

    # contraste
    enhancer = ImageEnhance.Contrast(imagen)
    imagen = enhancer.enhance(3)

    imagen = imagen.resize((28, 28))

    transform = T.Compose([
        T.ToTensor(),
        T.Normalize((0.1307,), (0.3081,)) #como mnist
    ])
    tensor = transform(imagen)
    tensor = tensor.unsqueeze(0)
    return tensor


## Clasificacion


In [ ]:
numeros= ["0.jpg", "1.jpeg", "2.jpeg", "3.jpeg", "4.jpg", "5.jpg", "6.jpg", "7.jpg", "8.jpg", "9.jpg"]
for i in numeros:
    imagen_tensor = procesar_imagen(i)
    z, y_pred = encoder(imagen_tensor.to(device))
    clase = torch.argmax(y_pred, dim=1).item()
   # print(imagen_tensor.min(), imagen_tensor.max(), imagen_tensor.mean())
    print(f"Número predicho por la red: {clase}")

Número predicho por la red: 0
Número predicho por la red: 1
Número predicho por la red: 2
Número predicho por la red: 3
Número predicho por la red: 4
Número predicho por la red: 2
Número predicho por la red: 5
Número predicho por la red: 1
Número predicho por la red: 5
Número predicho por la red: 3


# DISCUSION

Luego de 150 épocas, se logró una precisión del 80% sobre un conjunto de evaluación.

La red fue probada con dígitos dibujados a mano, alcanzando una precisión cercana al 50%. Mostró buen desempeño en la detección de los dígitos del 0 al 4, pero presentó dificultades con el 5, 6, 7 y 8.

No se alcanzó un equilibrio de Nash entre el encoder y los discriminadores: en todos los casos, el encoder aprendió a generar vectores Y representativos, pero rara vez logró producir vectores Z lo suficientemente similares a los reales como para engañar al discriminador.

# CONCLUSIONES

En este trabajo se desarrolló un Autoencoder Adversarial (AAE) para clasificación semi-supervisada del dataset MNIST, utilizando únicamente 200 ejemplos etiquetados. A pesar de la escasez de datos supervisados, se logró una precisión del 80% sobre el conjunto de testing, lo cual valida el potencial de los modelos generativos adversariales para tareas con bajo acceso a etiquetas.

El diseño del AAE se basó en dividir la representación latente en dos componentes:
z: codifica el estilo(información no supervisada)
y: codifica la clase(etiqueta supervisada)

Durante el entrenamiento, el encoder aprendió efectivamente a generar representaciones y útiles para la clasificación. Sin embargo, la parte no supervisada z no logró aproximar adecuadamente la distribución gaussiana deseada, y el discriminador correspondiente detectó fácilmente que los vectores z eran generados. Esto sugiere que no se alcanzó un equilibriopleno, y que el espacio latente no fue regularizado de la mejor manera.

A nivel teórico, esto se puede interpretar como una falla en lograr un equilibrio de Nash Nash entre el encoder y los discriminadores. En un AAE bien entrenado, el encoder debería ser capaz de engañar al discriminador , haciendo que sus salidas sean indistinguibles de muestras reales. Si esto no ocurre, el modelo puede sobreajustarse al objetivo de clasificación y perder la riqueza del espacio latente, lo cual afecta la generalización, en especial cuando se evalúa con datos fuera de distribución, como dígitos dibujados a mano.

Esta experiencia pone de manifiesto tanto el potencial como las dificultades prácticas de entrenar modelos adversariales en entornos semi-supervisados. La arquitectura AAE es conceptualmente elegante, pero altamente sensible al equilibrio entre sus componentes. Abordar este desafío abre puertas no solo a mejores clasificadores, sino también a modelos generativos que comprendan de forma más profunda la estructura de los datos.


Líneas de trabajo futuras y posibles mejoras a la red:

Agregar capas convolucionales: Las redes convolucionales son especialmente efectivas en visión computacional, ya que aprovechan la estructura espacial de las imágenes. Reemplazar las capas densas del encoder y decoder por capas convolucionales podría mejorar la extracción de características locales y reducir la sensibilidad a trazos y estilos personales.

Preentrenar el autoencoder sin adversarialidad: Entrenar primero el autoencoder solo con la reconstrucción podría permitir que el modelo aprenda una representación inicial más estable, y luego introducir el entrenamiento adversarial de manera progresiva.

Aplicar data augmentation: Esto es especialmente relevante si se pretende usar el modelo con imágenes dibujadas por humanos. Aumentar la variedad durante el entrenamiento (rotaciones, traslaciones, ruido) puede robustecer la red frente a variaciones de estilo

Mejorar la estrategia de selección de datos etiquetados: En lugar de elegir al azar, pero podría explorarse una selección informada que garantice una mejor cobertura de las distintas clases y estilos.



# REFERENCIAS

Makhzani, A., Shlens, J., Jaitly, N., Goodfellow, I., & Frey, B. (2016, mayo 25). Adversarial autoencoders. arXiv.
https://arxiv.org/abs/1511.05644


In [ ]:
# Guardar los pesos del encoder
torch.save(encoder.state_dict(), "encoder_final.pth")
torch.save(decoder.state_dict(), "decoder_final.pth")
torch.save(dz.state_dict(), "dz_final.pth")
torch.save(dy.state_dict(), "dy_final.pth")


In [ ]:
# Definí la arquitectura tal como en el entrenamiento
encoder = Encoder().to(device)

# Cargar pesos entrenados
encoder.load_state_dict(torch.load("encoder_final.pth"))
encoder.eval()
